# Text Generation with Flan-T5 XL and Flan-T5 XXL on IPUs

[Flan-T5](https://huggingface.co/docs/transformers/model_doc/flan-t5) is an encoder-decoder transformer model that reframes all NLP tasks into a text-to-text format. Compared to T5, Flan-T5 has been fine-tuned on more than 1,000 additional tasks.

Language models are very powerful because a huge variety of tasks can be formulated as text-to-text problems and thus adapted to fit the generative setup, where the model is asked to correctly predict future tokens. For more details, check out the T5 paper [Exploring the Limits of Transfer Learning with a Unified Text-to-Text Transformer](https://arxiv.org/pdf/1910.10683.pdf) and the Flan-T5 paper [Scaling Instruction-Finetuned Language Models](https://arxiv.org/pdf/2210.11416.pdf).

In this notebook, we will demonstrate how you can use Flan-T5 XL and Flan-T5 XXL on readily available IPU-POD4 and IPU-POD16 systems, respectively, for for common NLP workloads.

|  Domain | Tasks | Model | Datasets | Workflow |   Number of IPUs   | Execution time |
|---------|-------|-------|----------|----------|--------------|--------------|
|   NLP   |  Few-shot text generation  | Flan-T5 | N/A | Inference | recommended: 16 (min: 4) |  13 min (with 4 IPUs: 5 min)  |

In this notebook, you will:
- Generate text in five lines of code with Flan-T5 XXL on the Graphcore IPU
- Improve text generation throughput using batched inference
- Use Flan-T5 XXL to answer questions and build prompts to reliably use text-generation for more specific NLP tasks
- Explore the capabilities of the model with zero-shot (no examples included in the prompt) and few-shot prompting (where a few typical examples are included in the prompt)

By exploring this notebook, you will gain insight into how various tasks can be formulated in a text-to-text format and how this flexibility can be utilized to fine-tune the model using your own dataset.

No fine-tuning is performed in this notebook. You can learn more about fine-tuning Flan-T5 in the [Textual Entailment on IPUs using Flan-T5 - Fine-tuning](Flan-T5-textual-entailment-fine-tuning.ipynb) notebook.

[![Join our Slack Community](https://img.shields.io/badge/Slack-Join%20Graphcore's%20Community-blue?style=flat-square&logo=slack)](https://www.graphcore.ai/join-community)

## Environment setup

The best way to run this demo is on Paperspace Gradient's cloud IPUs because everything is already set up for you.

[![Run on Gradient](../../../gradient-badge.svg)](https://ipu.dev/Os21ny)

To run the demo using other IPU hardware, you need to have the Poplar SDK enabled. Refer to the [Getting Started guide](https://docs.graphcore.ai/en/latest/getting-started.html#getting-started) for your system for details on how to do this. Also refer to the [Jupyter Quick Start guide](https://docs.graphcore.ai/projects/jupyter-notebook-quick-start/en/latest/index.html) for how to set up Jupyter to be able to run this notebook on a remote IPU machine.

## Dependencies and configuration

Install the dependencies for this notebook.

In [ ]:
%pip install -r requirements.txt

## Initial setup

In [ ]:
import os

number_of_ipus = int(os.getenv("NUM_AVAILABLE_IPU", 16))
number_of_ipus

In [ ]:
model_size = "xl" if number_of_ipus == 4 else "xxl"

This notebook supports both Flan-T5 XXL (11B parameters) and Flan-T5 XL (3B parameters). The code below refers to the XXL model, which needs a minimum of 16 IPUs to run. Handling the weights and 🤗 checkpoints is going to take a bit more time compared to the XL variant, which needs a minimum of 8 IPUs to run. If you want to run quicker experiments and model performance is not a priority right now, you can change the code below to use the XL model.

In [ ]:
# Uncomment the following line if you want to use the smaller variant
# model_size = "xl"

In [ ]:
import os

ipus_needed = 16 if model_size == "xxl" else 4
if number_of_ipus < ipus_needed:
    raise ValueError(
        f"This example needs {ipus_needed} IPUs to work. Detected {number_of_ipus}"
    )

os.environ["POPART_CACHE_DIR"] = os.getenv(
    "POPLAR_EXECUTABLE_CACHE_DIR", "./exe_cache/"
)

## Running Flan-T5 on the IPU

In a few lines of code, we load the configuration and use it to create a pipeline object which will allow us to interactively run the model and use it for text generation on Graphcore IPUs. Base configurations are available in the `config/inference.yml` file and can be loaded as follows:

In [ ]:
from utils.setup import t5_config_setup
from config import CONFIG_DIR

config, *_ = t5_config_setup(CONFIG_DIR / "inference.yml", "release", model_size)
print(config.dumps_yaml())

The configuration can be edited and saved in a new file to suit your needs. It contains all the arguments which define the model and controls the execution of the application on the IPU.

For inference, the main arguments we are interested in modifying are:

In [ ]:
# The following values for the mini batch size are the maximum that can fit in device memory
micro_batch_size = 3 if model_size == "xxl" else 4
# The number of prompts which will be processed at once
config.execution.micro_batch_size = micro_batch_size
# The maximum sequence length handled by the model
config.model.sequence_length = 512
# The number of tokens generated before stopping
# Note: the model will stop before this if it generates an </s> (end-of-text) token
config.inference.output_length = 20

Next, we're going to combine this configuration with pre-trained weights. The Flan-T5 checkpoints are available on 🤗 at [google/flan-t5-xxl](https://huggingface.co/google/flan-t5-xxl) and [google/flan-t5-xl](https://huggingface.co/google/flan-t5-xl).
The pipeline utility directly accepts the name of a pre-trained checkpoint from the 🤗 Hub.

Once we have a config and have chosen a pre-trained model we create a `T5Pipeline` pipeline. If you are not planning to use long prompts, you can reduce the sequence length either by changing the config or by providing the `sequence_length` argument to the pipeline. Reducing the sequence length allows you to fit more batches into memory. As you will see in the batched inference section, you can maximise performance by making the model process several prompts at the same time.

Creating the pipeline takes a few minutes as the checkpoint is downloaded and loaded into the session:

In [ ]:
import api

flan_t5 = api.T5Pipeline(
    config,
    model_size,
    f"google/flan-t5-{model_size}",
    sequence_length=512,
    micro_batch_size=micro_batch_size,
    output_length=20,
    print_live=True,
)

You can explore the attributes of the `general_model` pipeline:

- `tokenizer` contains the tokenizer loaded with the pre-trained checkpoint from the 🤗 Hub;
- `config` has the configuration we prepared;
- `session` is the session which is used to run the model on the IPU.

You can use the pipeline to do standard text generation starting from an input prompt. In this demo, we use **greedy generation**, meaning the highest probability token is chosen at each step. This corresponds to running a 🤗 pipeline with the option `sample=False`.
The first execution of the pipeline takes a few minutes because the weights are copied to the IPU. Subsequent executions will be much faster.
In the cell below, we ask the model to answer a simple question:

In [ ]:
out = flan_t5("What is the capital of Italy?")

The answer is correct, and the model includes the `</s>` token, marking the end of the generated text.

## Batched text generation

This model configuration supports batched generation where we can generate text based on multiple prompts at the same time. This can be useful to increase throughput when text prompts are queued for processing in a production environment. This config has a batch size of 3 which means this pipeline can generate answers to three questions at a time. However, we can pass a list of more than three prompts and the pipeline object will simply process them all in batches of three.

Let's test the model on various tasks in a batch:

In [ ]:
prompts = [
    "Translate to Italian: I love pizza.",
    "Q: Can Spider-Man have a conversation with Elon Musk? Give the rationale before answering.",
    "Please answer the following question. What is the freezing point of Oxygen?",
    "A step-by-step recipe to cook pesto pasta:",
    "Solve the following equation for x: x^2 - 9 = 0",
    "Premise: At my age you will probably have learnt one lesson. Hypothesis: It's not certain how many lessons you'll learn by your thirties. Does the premise entail the hypothesis?",
]

In [ ]:
for out in flan_t5(prompts, print_live=False, output_length=40):
    print(out)

> Note that some of these answers may be wrong, because information retrieval from the model itself is not the purpose of Flan-T5. However, if you use Flan-T5 XXL answers are less likely to be wrong compared to Flan-T5 XL.

## What can I use Flan-T5 for?

Flan-T5 has been fine-tuned on thousands of different tasks across hundreds of datasets. So no matter what your task might be, it's worth seeing if Flan-T5 can meet your requirements. Here we will look at a few of the common ones. We'll use a mixture of zero-shot and few-shot prompting, depending on the task.

The NLP tasks we explore below only scratch the surface of all the possibilities where this powerful model could be used. As mentioned, Flan-T5 has been fine-tuned on more than 1,000 different tasks. So, feel free to further explore the capabilities of Flan-T5 XXL.

### Sentiment analysis:

Sentiment analysis is a classification task, where the model needs to determine whether the sentiment expressed by a piece of text (for example, a review) is positive, negative or neutral.

In [ ]:
sentiment_analysis = """Review: The hotel room was clean and comfortable, and the view from the balcony was breathtaking. Sentiment: Positive
Review: This restaurant serves delicious food and the staff is very friendly. Sentiment: Positive
Review: The product arrived damaged and the company refused to replace it. Sentiment: Negative
Review: The book was a bit boring and the characters were poorly developed. Sentiment:"""
flan_t5(sentiment_analysis)[0]

### Advanced named entity recognition

This task consists of identifying key information in the given text, and returning it in the same format as that of the other examples provided. The exact entities to extract (and possibly classify) can vary depending on the use case. The following is an example.

In [ ]:
advanced_ner = """Google LLC, commonly known as Google, is a multinational technology company. Larry Page and Sergey Brin created Google in 1998."
[Company]: Google LLC
[Founders]: Larry Page, Sergey Brin

Tesla, Inc. is an electric vehicle and clean energy company. Elon Musk is the CEO and co-founder of Tesla, which was established in 2003."
[Company]: Tesla, Inc.
[Founders]: Elon Musk

Netflix, Inc. is an online streaming platform. Reed Hastings and Marc Randolph established Netflix in 1997."""
flan_t5(advanced_ner)[0]

### Question answering

The model is provided with a piece of text as context, followed by a question related to the text. The objective is to answer the question, extracting relevant information from the context.

The following example is from the [squad](https://huggingface.co/datasets/squad) validation dataset.

In [ ]:
context = """Philosophers in antiquity used the concept of force in the study of stationary and moving objects and simple machines,
but thinkers such as Aristotle and Archimedes retained fundamental errors in understanding force.
In part this was due to an incomplete understanding of the sometimes non-obvious force of friction,
and a consequently inadequate view of the nature of natural motion. A fundamental error was the belief that a force is required to maintain motion,
even at a constant velocity. Most of the previous misunderstandings about motion and force were eventually corrected by Galileo Galilei and Sir Isaac Newton.
With his mathematical insight, Sir Isaac Newton formulated laws of motion that were not improved-on for nearly three hundred years. By the early 20th century,
Einstein developed a theory of relativity that correctly predicted the action of forces on objects with increasing momenta near the speed of light,
and also provided insight into the forces produced by gravitation and inertia."""
question = "How long did it take to improve on Sir Isaac Newton's laws of motion?"
# The correct answer is nearly three hundred years
flan_t5(f"{context} {question}")[0]

### Intent classification

In this task, the model needs to identify what the intent expressed by the given text is.

In [ ]:
intent_classification = """[Text]: What time does the movie start tonight?
[Intent]: inquire about movie time

[Text]: I want to book a table for two at a restaurant for tomorrow evening.
[Intent]: make restaurant reservation

[Text]: Is there a nearby pharmacy that's open 24/7?
[Intent]:"""
flan_t5([intent_classification])[0]

### Summarization

The objective of the model is to summarise a potentially large piece of text that is provided.

The following example is from the [xsum](https://huggingface.co/datasets/xsum) validation dataset.

In [ ]:
summarization = """Document: A team from America have looked at how ants have evolved and found that the insects had complex underground farms,
growing different types of fungi. Not only that, the ants had taken fungus from its natural environment to nests in much drier climates.
The fungus couldn't survive without the ants so it is a very early example of agriculture. Agriculture is what we call the practice of farming.
Such as preparing soil for growing crops and raising animals for food and other products such as wool. Dr Ted Schultz who specialises in ants
and works at the Museum of Natural History said that "Ants have established a form of agriculture that provides all the nourishment needed for
their societies using a single crop that is resistant to disease, pests and droughts at a scale and level of efficiency that rivals human agriculture."
There are many species of farmer ant, over 250 have been found in tropical forests, deserts and grasslands in the Americas and the Caribbean.
Summary:"""
# The correct answer should be something like the following:
# "Scientists have found out that ants began farming millions of years before humans did."
flan_t5(summarization)[0]

### Text classification

This task is to classify the input text into categories. The exact categories to use can vary depending on the use case. In the following examples, we ask the model to determine the topic and the keywords, respectively.

In [ ]:
text_classification_1 = """"Learning to play the guitar starts with mastering the basic chords. Once you have a solid foundation, you can progress to more advanced techniques.
topic: guitar learning

Regular exercise is crucial for maintaining physical fitness and overall well-being. It helps improve cardiovascular health and boosts mood.
topic: fitness benefits

In computer programming, object-oriented programming (OOP) allows for the creation of reusable code through the use of classes and objects.
topic:"""
# The correct answer should be something like "object oriented programming"
flan_t5(text_classification_1)[0]

In [ ]:
text_classification_2 = """Flan-T5 was released in the paper Scaling Instruction-Finetuned Language Models - it is an enhanced version of T5 that has been finetuned in a mixture of tasks.
keywords: released, enhanced, finetuned

The IPU, or Intelligence Processing Unit, is a highly flexible, easy-to-use parallel processor designed from the ground up for AI workloads.
keywords: processor, AI

Paperspace is the platform for AI developers. providing the speed and scale needed to take AI models from concept to production.
keywords:"""
# The correct answer could include keywords like "AI, speed, scale, platform, production"
flan_t5(text_classification_2)[0]

## (Optional) Release IPU resources

When you're done using Flan-T5, free up IPU resources by running:

In [ ]:
flan_t5.detach()

## Conclusion

This notebook has demonstrated how easy it is to run Flan-T5 XXL on Graphcore IPUs using this implementation of the model and 🤗 Hub checkpoints of the model weights. While not as powerful as larger models for free text generation, medium-sized models like Flan-T5 can still be used for a wide range of NLP tasks. We've seen the model in action on sentiment analysis, named entity recognition, question answering, intent classification, summarisation, and text classification.

In less than 10 lines of code, we were able to load the model onto the IPU and perform NLP tasks. We've seen how batched inference can be used to increase throughput by generating multiple answers at a time. We built structured prompts for various NLP tasks using few-shot prompting, concatenating several examples in the input prompt, to give the model instructions for the task. We also tried zero-shot prompting where no examples were included in the input prompt, showing that the model can perform well in both cases.

In order for Flan-T5 to perform well on some tasks, it might require fine-tuning. If you'd like to learn more about fine-tuning Flan-T5 XXL, check out the [Textual Entailment on IPUs using Flan-T5 - Fine-tuning](Flan-T5-textual-entailment-fine-tuning.ipynb) notebook, where we show that the fine-tuned model can achieve an accuracy of about 87% on the MNLI GLUE dataset.

If you'd like to try inference with Flan-T5 Large, or do inference faster on IPU-POD16 with Flan-T5 XL, take a look at the [Flan-T5-Large/XL inference notebook](https://ipu.dev/tvxZ3Q).